In [6]:
# ============================================================
# 🌳 Hierarchie-Baum mit Filter + Excel-Export
# ============================================================

# ----------------------------
# 1️⃣ Bibliotheken
# ----------------------------
import pandas as pd
from collections import defaultdict

import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

from openpyxl.utils import get_column_letter


# ----------------------------
# 2️⃣ CSV einlesen
# ----------------------------
CSV_FILE = "20250318_Export_KUBA_EPZ-KernInfO_DE_FR.csv"

df = pd.read_csv(
    CSV_FILE,
    sep=";",
    engine="python",
    dtype=str   # wichtig für Hierarchiecodes
)


# ----------------------------
# 3️⃣ Spaltenindexe (0-basiert)
# ----------------------------
HIER_CODE_COL = 4      # Hierarchiecode
TYPE_NAME_COL = 5      # Typname
OBJ1_COL = 1           # Objektbezeichnung 1
OBJ2_COL = 2           # Objektbezeichnung 2
FILTER_COL = 21        # Optionales Filterattribut


# ----------------------------
# 4️⃣ Baum & Counts erzeugen
# ----------------------------
def build_tree_from_df(df_filtered):
    tree = defaultdict(list)
    agg_counts = defaultdict(int)
    direct_counts = defaultdict(int)
    code_to_objects = defaultdict(list)
    code_to_name = {}

    all_codes = set()

    # 1️⃣ Alle Codes + Eltern sammeln (Struktur)
    for _, row in df_filtered.iterrows():
        code = str(row.iloc[HIER_CODE_COL]).strip()
        if not code or code == "nan":
            continue

        name = str(row.iloc[TYPE_NAME_COL]).strip()
        code_to_name[code] = name

        # alle Hierarchieebenen sammeln
        for i in range(1, len(code) + 1):
            all_codes.add(code[:i])

    # 2️⃣ Baumstruktur aufbauen (auch ohne Counts)
    for code in all_codes:
        if len(code) > 1:
            parent = code[:-1]
            if code not in tree[parent]:
                tree[parent].append(code)

    # 3️⃣ Counts & Objekte zählen
    for _, row in df_filtered.iterrows():
        code = str(row.iloc[HIER_CODE_COL]).strip()
        if not code or code == "nan":
            continue

        obj = f"{row.iloc[OBJ1_COL]} / {row.iloc[OBJ2_COL]}"
        direct_counts[code] += 1
        code_to_objects[code].append(obj)

        # aggregierte Zählung
        for i in range(1, len(code) + 1):
            agg_counts[code[:i]] += 1

    return tree, agg_counts, direct_counts, code_to_objects, code_to_name

# ----------------------------
# 5️⃣ Button: direkte Objekte
# ----------------------------
def direct_objects_button(code, code_to_objects, code_to_name):
    objs = code_to_objects.get(code, [])
    if not objs:
        return None

    btn = widgets.Button(
        description=f"📄 Direkte Einträge ({len(objs)})",
        layout=widgets.Layout(width="auto")
    )

    def on_click(b):
        display(Markdown(
            f"### Direkte Objekte für {code} – {code_to_name.get(code,'')}\n"
            + "\n".join(f"- {o}" for o in objs)
        ))

    btn.on_click(on_click)
    return btn


# ----------------------------
# 6️⃣ Baum-Widget (rekursiv)
# ----------------------------
def create_tree_widget(code, code_to_objects, tree, agg_counts, direct_counts, code_to_name):
    agg = agg_counts.get(code, 0)
    direct = direct_counts.get(code, 0)
    name = code_to_name.get(code, "")

    header = widgets.HTML(
        value=f"<b>{code}</b> – {name} "
              f"<span style='color:gray'>({agg} / {direct})</span>"
    )

    elements = [header]

    btn = direct_objects_button(code, code_to_objects, code_to_name)
    if btn:
        elements.append(btn)

    children = sorted(tree.get(code, []))
    if children:
        child_widgets = [
            create_tree_widget(c, code_to_objects, tree, agg_counts, direct_counts, code_to_name)
            for c in children
        ]
        acc = widgets.Accordion(children=child_widgets)
        for i, c in enumerate(children):
            acc.set_title(i, c)
        elements.append(acc)

    return widgets.VBox(elements)


# ----------------------------
# 7️⃣ Hierarchisch sortierte Tabelle
# ----------------------------
def extract_hierarchy_table(tree, code_to_name):
    rows = []

    all_codes = set(tree.keys())
    for children in tree.values():
        all_codes.update(children)

    # Roots = Codes ohne Parent
    roots = sorted(
        c for c in all_codes
        if len(c) == 1 or c[:-1] not in all_codes
    )

    def walk(code, parent=""):
        rows.append({
            "Ebene": len(code),
            "Code": code,
            "Name": code_to_name.get(code, ""),
            "Parent": parent
        })

        for child in sorted(tree.get(code, [])):
            walk(child, code)

    for r in roots:
        walk(r)

    return pd.DataFrame(rows)


# ----------------------------
# 8️⃣ Excel-Export (schön formatiert)
# ----------------------------
def export_hierarchy_to_excel(df_hierarchy, filename="Haupthierarchie.xlsx"):
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        df_hierarchy.to_excel(
            writer,
            index=False,
            sheet_name="Hierarchie"
        )

        ws = writer.sheets["Hierarchie"]
        ws.freeze_panes = "A2"
        ws.auto_filter.ref = ws.dimensions

        for col in range(1, ws.max_column + 1):
            ws.column_dimensions[get_column_letter(col)].width = 24


# ----------------------------
# 9️⃣ Filter-Dropdown
# ----------------------------
filter_values = sorted(df.iloc[:, FILTER_COL].dropna().unique().tolist())

filter_dropdown = widgets.Dropdown(
    options=["Alle"] + filter_values,
    description="Filter:"
)

output = widgets.Output()
display(filter_dropdown, output)


# ----------------------------
# 🔟 Callback: Baum + Export
# ----------------------------
def on_filter_change(change):
    with output:
        clear_output()

        if change["new"] == "Alle":
            df_f = df
        else:
            df_f = df[df.iloc[:, FILTER_COL] == change["new"]]

        tree, agg, direct, objects, names = build_tree_from_df(df_f)

        display(Markdown("## 🌳 Hierarchie-Baum"))
        roots = sorted([c for c in agg if len(c) == 2])
        for r in roots:
            display(create_tree_widget(r, objects, tree, agg, direct, names))

        df_hierarchy = extract_hierarchy_table(tree, names)
        export_hierarchy_to_excel(df_hierarchy)

        display(Markdown("📊 **Haupthierarchie wurde als Excel exportiert**"))


# ----------------------------
# ▶️ Start
# ----------------------------
filter_dropdown.observe(on_filter_change, names="value")
on_filter_change({"new": "Alle"})


Dropdown(description='Filter:', options=('Alle', 'Geplant\xa0(vor\xa0PGV)', 'In\xa0Bau', 'In\xa0Betrieb', 'Pro…

Output()